# D-Hydro Boezemmodel V4 - Modeltests

Met deze notebook tonen we de prestaties van het D-Hydro model van Noorderzijlvest. 

Deze notebook is opgezet door Daniel Tollenaar (D2Hydro), Vincent de Looij (Waterschap Noorderzijlvest) en Siebe Bosch (HydroConsult).

We tonen hieronder de resultaten van een drietal tests:
1. <br>Peil-in-rust:</br> de leegloop van het model na een intialisatie met een hoge waterdiepte. Afwijkingen ten opzichte van streefpeil leiden naar lekken of obstructies in de modelschematisatie
2. <br>Half-maatgevende-afvoer:</br> de waterstanden en verhanglijnen na een stationaire som met een afvoer van 7 mm/dag. Afwijkingen ten opzichte van insteekhoogte leiden naar knijpende constructies of waterlopen in de modelschematisatie.
3. <br>Dynamische afvoer:</br> een niet-stationaire extreme neerslag die nog (net) niet tot overlast moet leiden. Afwijkingen ten opzichte van insteekhoogte leiden naar knijpende constructies of waterlopen in de modelschematisatie.

Bij alle tests is ook gekeken naar de rekenkundige prestaties; numerieke stabiliteit en de daaraan gekoppelde rekentijd.

Met deze notebook kunt u zelf meezoeken. Alle resultaten worden weergegeven in interactieve plots.

In [1]:
import netCDF4 as nc
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, Vendors
from bokeh.models import ColumnDataSource, HoverTool, RangeSlider, Slider, CustomJS
from bokeh.layouts import grid

output_notebook(hide_banner=True)
tile_provider = get_provider(Vendors.CARTODBPOSITRON)

data_path = Path(r".\data").absolute().resolve()
beheerregister = data_path.joinpath("beheerregister")

invoerbestanden = {
    "peilgebieden": "Peilgebieden.shp"
}

modeltestbestanden = Path("modeltests")

## Peil In Rust

Het peil-in-rust (PIR) is als volgt berekend:
- Een initiele waterdiepte van 6m over het hele netwerk
- Een simulatieperiode van 10 dagen

Deze simulatie is uitgevoerd op een laptop in ongeveer 50 minuten.

### Zomer

In [2]:
ds = nc.Dataset(r".\modellen\boezemmodel\fm\output_initialisatie\boezemmodel_map.nc")
gpg_gdf = gpd.read_file(beheerregister.joinpath(invoerbestanden["peilgebieden"]))
ref_level = "OPVAFWZP"

# inlezen waterhoogtes uit NetCDF
head_gdf = gpd.GeoDataFrame(
    data={"water_level": ds["mesh1d_s1"][-1],
          "geometry": [
              Point(coords) for coords in zip(ds["mesh1d_node_x"], ds["mesh1d_node_y"])
              ]}
    )

head_gdf.crs = "epsg:28992"
head_gdf["wl_diff"] = 0

# verschil met zomerpeil berekenen
for _, row in gpg_gdf.iterrows():
    zomer_peil = row[ref_level]
    set_idx = head_gdf.loc[
        head_gdf["geometry"].within(row["geometry"])
        ].index.to_list()
    if set_idx:
        head_gdf.loc[set_idx, ("zomer_peil")] = zomer_peil
        head_gdf.loc[set_idx, ("wl_diff")] = head_gdf.loc[
            set_idx]["water_level"
                     ] - zomer_peil

head_gdf.to_file(modeltestbestanden.joinpath("peil_in_rust_zomer.shp"))

# afvoer uit NetCDF
q_gdf = gpd.GeoDataFrame(
    data={"discharge": ds["mesh1d_q1"][-1],
          "geometry": [
              Point(coords) for coords in zip(ds["mesh1d_edge_x"], ds["mesh1d_edge_y"])
              ]}
    )

q_gdf.crs = "epsg:28992"


q_gdf.to_file(modeltestbestanden.joinpath("afvoer_in_rust_zomer.shp"))

#### Afvoer
Bij het PIR dient de afvoer overal +/- 0 m3/s te zijn. Het systeem is immers in rust. Bij onderstaande grafiek kunt u onderzoeken in welke mate dit voor deze simulatie het geval is. Met de slider kunt u inzichtelijk maken welke gebieden meer of minder "in rust" zijn. Op de rekenpunten kunt u de waarden opvragen

In [3]:
p1 = figure()
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
gdf = q_gdf.to_crs("epsg:3857").copy()
gdf['x'] = gdf['geometry'].x
gdf['y'] = gdf['geometry'].y
df = gdf.drop('geometry', axis=1)
df.loc[:, "discharge"] = df["discharge"].abs()
df.to_pickle("afvoer.pickle")

q_slider = Slider(start=0.01,
                  end=max(df["discharge"].max(), 0.1),
                  value=0.05,
                  step=.01,
                  title="m3/s")


df["label"] = "tussen"
df["color"] = "white"
df.loc[df["discharge"] > q_slider.value, ["color", "label"]] = ["blue", "hoger"]
q_source = ColumnDataSource(df)

q_legend_callback = CustomJS(args=dict(source=q_source,
                                     ql=q_slider), code="""
    var nrows = source.get_length()
    var data = source.data;
    var q = data['discharge']
    var color = data['color']
    var label = data['label']
    for (var i = 0; i < nrows; i++) {
        if (q[i] > ql.value) {
          color[i] = "blue"
          label[i] = "hoger";
        } else {
          color[i] = "white"
          label[i] = "lager";
        }
    }
    source.change.emit();
""")

tooltips=[('afvoer [m3/s]', '@discharge{0.2f}')]

p1.add_tile(tile_provider, name="background")
p1.circle(source=q_source, fill_color='color', legend_field='label', line_alpha=0)
p1.add_tools(HoverTool(tooltips=tooltips))

q_slider.js_on_change("value", q_legend_callback)

show(grid([q_slider,p1]))

#### Peil
Bij het PIR dient de het gesimuleerde peil en rust overal ongeveer gelijk te zijn aan de streefpeilen. Bij onderstaande grafiek kunt u onderzoeken in welke mate dit voor deze simulatie het geval is. Met de slider kunt u inzichtelijk maken welke gebieden meer of minder afwijking hebben ten opzichte van peil. Op de rekenpunten kunt u de waarden opvragen

In [4]:
p2 = figure()
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
gdf = head_gdf.to_crs("epsg:3857").copy()
gdf['x'] = gdf['geometry'].x
gdf['y'] = gdf['geometry'].y
df = gdf.drop('geometry', axis=1)
df.to_pickle("waterstand.pickle")

low_level_slider = Slider(start=min(df["wl_diff"].min(), -0.1),
                               end=-0.01,
                               value=-0.1,
                               step=.01,
                               title="<0m")

high_level_slider = Slider(start=0.01,
                                end=max(df["wl_diff"].max(), 0.1),
                                value=0.1,
                                step=.01,
                                title=">0m")

df["label"] = "tussen"
df["color"] = "white"
df.loc[df["wl_diff"] < low_level_slider.value, ["color", "label"]] = ["red", "lager"]
df.loc[df["wl_diff"] > high_level_slider.value, ["color", "label"]] = ["blue", "hoger"]
source = ColumnDataSource(df)

legend_callback = CustomJS(args=dict(source=source,
                                     lr=low_level_slider,
                                     hr=high_level_slider), code="""
    var nrows = source.get_length()
    var data = source.data;
    var diff = data['wl_diff']
    var color = data['color']
    var label = data['label']
    for (var i = 0; i < nrows; i++) {
        if (diff[i] < lr.value) {
          color[i] = "red"
          label[i] = "lager";
        } else if (diff[i] > hr.value) {
          color[i] = "blue"
          label[i] = "hoger";
        } else {
          color[i] = "white"
          label[i] = "tussen";
        }
    }
    source.change.emit();
""")

tooltips=[('peil [m NAP]', '@zomer_peil{0.2f}'),
          ('berekend [m NAP]:', '@water_level{0.2f}'),
          ('verschil [m]', '@wl_diff{0.2f}')]

p2.add_tile(tile_provider, name="background")
p2.circle(source=source, fill_color='color', legend_field='label', line_alpha=0)
p2.add_tools(HoverTool(tooltips=tooltips))

low_level_slider.js_on_change("value", legend_callback)
high_level_slider.js_on_change("value", legend_callback)

show(grid([[low_level_slider,high_level_slider],p2]))